### Facsimile inversion 

In [ ]:
import numpy as np
import itertools as it
# from Comm.utils import iterative_solve_eta as itsol
import sys
sys.path.append('../Comm/')
from utils import iterative_solve_eta as itsol
import matplotlib.pyplot as plt
from scipy.special import softmax

torch_available = False
try :
    import torch as torch
    dtype = torch.double
    if torch.cuda.is_available():
        device = torch.device("cuda:0")
        print('Device: GPU')
    else:
        device = torch.device("cpu")
        print('Device: CPU')
    torch_available = True
    print('Pytorch available')
except ImportError:
    print('Pytorch not available')

petsc4py_available = False
try :
    import petsc4py
    petsc4py.init()
    from petsc4py import PETSc
    petsc4py_available = True
    print('Petsc4py available')
except ImportError:
    print('Petsc4py not available')

scipy_sparse_available = False
try :
    import scipy.sparse as sparse
    scipy_sparse_available = True
    print('Scipy sparse available')
except ImportError:
    print('Scipy sparse not available')

cupy_scipy_available = False
try : 
    import cupy as cp
    import cupyx.scipy.sparse as cusparse
    import cupyx.scipy.sparse.linalg
    cupy_scipy_available = True
    print('Cupy sparse available')
except ImportError:
    print('Cupy sparse not available')

### Tip
To make available the module petsc4py in the Ullyses cluster do the following:
```
module load gnu8 openmpi3 openblas scalapack cuda/11.8 phdf5 python3/3.8
pip install --user Cython==0.29.32
pip install --user mpi4py
export PETSC_CONFIGURE_OPTIONS=' --with-cuda=1 --with-debugging=0'
pip install --user petsc # this will take a while
pip install --user petsc4py
```

### Main problem 
Solve the following:
$$
\begin{align*}
(\mathbf{1} - \gamma \mathbf{T})  \eta &=  \rho
\end{align*}
$$
### Variables
$$
\begin{align*}
\text{pi} &\to \pi(a\ m'\ |\ m\ y)\\
\text{PObs\char`_lim} &\to f(y\ |\ m\ s)\\
\text{gamma} &\to \gamma \to \text{Discount factor}\\
\text{rho0} &\to \rho \to \text{Distribution of the initial position of the agent}\\
\text{Lx, Ly, Lx0, Ly0} &\to \text{Environment setting}\\
\text{find\char`_range} &\to \text{Radius threshold for reach the price}
\end{align*}
$$

## Utilities functions

In [ ]:
clip = lambda x, l, u: l if x < l else u if x > u else x

def index_six_to_two(index,M,Ly,Lx):
    # Convert the index of an array of 6 dim to 2 dim
    new_index_x = index[0] * Lx * Ly + index[1] * Lx + index[2]
    new_index_y = index[3] * Lx * Ly + index[4] * Lx + index[5]

    return (new_index_x,new_index_y)

def solution_test(eta,T,rho,gamma,plot=False):
    sol = (np.eye(T.shape[0]) - gamma * T) @ eta
    if plot:
        # title of the plot rho - (1 - gamma * T) @ eta put in latex style
        plt.title(r'$\mathbf{\rho} - (\mathbf{1} - \gamma \mathbf{T}) \mathbf{\eta} = 0 $')
        plt.plot(sol-rho)
        plt.show()
    return np.allclose(sol,rho)
    

## Build Tsm_sm matrix

In [ ]:
def build_Tsm_sm_dense(M,Lx,Ly,Lx0,Ly0,find_range,action_size,p_a_mu_m_xy):
    # Action Order
    # left, right, up, down
    
    # tuples to be populated
    #      T indices : "left / right / up / down"
    #      (m', y', x', m, y, x)
    #      pi indices : "left_act / ... "
    #      (m', a, m, y, x)

    # Create the matrix
    Tsm_sm = np.zeros( (M, Ly, Lx, M, Ly, Lx) )

    # Fill the matrix
    for im_new in range(M):
        
        left = [ (im_new, iy, clip(ix-1, 0, Lx-1), im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]
        left_act = [ (0, im_new, im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]
        
        for l, la in zip(left, left_act):
            Tsm_sm[l] += p_a_mu_m_xy[la]
        
        right = [ (im_new, iy, clip(ix+1, 0, Lx-1), im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]
        right_act = [ (1, im_new, im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]
        for r, ra in zip(right, right_act):
            Tsm_sm[r] += p_a_mu_m_xy[ra]
    
        up = [ (im_new, clip(iy+1, 0, Ly-1), ix,  im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]
        up_act = [ (2, im_new, im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]
        for u, ua in zip(up, up_act):
            Tsm_sm[u] += p_a_mu_m_xy[ua]
        
        down = [ (im_new, clip(iy-1, 0, Ly-1), ix, im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]
        down_act = [ (3, im_new, im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]
        for d, da in zip(down, down_act):
            Tsm_sm[d] += p_a_mu_m_xy[da]
    
    # Delete the rows and columns that have been set to zero depending on the distance to the initial position
    yxs = it.product(np.arange(Ly), np.arange(Lx))
    yx_founds = it.filterfalse(lambda x: (x[0]-Ly0)**2 + (x[1]-Lx0)**2 > find_range**2, yxs)
    
    for yx_found in yx_founds:
        # all transitions starting from the source do not go anywhere
        Tsm_sm[:,yx_found[0],yx_found[1],:,:,:] = 0
        # all transitions ending in the source stop the episode
        Tsm_sm[:,:,:,:,yx_found[0],yx_found[1]] = 0
    
    Tsm_sm_matrix = np.reshape(Tsm_sm, (M*Ly*Lx, M*Ly*Lx))

    return Tsm_sm_matrix

def build_Tsm_sm_sparse(M,Lx,Ly,Lx0,Ly0,find_range,action_size,p_a_mu_m_xy):

    # Get the maximum number of non zeros values per row for Tsm_sm
    count_non_zeros_in_row = np.ones(M*Ly*Lx,dtype=int) * action_size * M
    count_non_zeros_in_row[0] -= M
    count_non_zeros_in_row[-1] -= M

    # Set the amount of non zero values to zero depending on the distance to the center
    yxs = it.product(np.arange(Ly), np.arange(Lx))
    yx_founds = it.filterfalse(lambda x: (x[0]-Ly0)**2 + (x[1]-Lx0)**2 > find_range**2, yxs)

    for iy, ix in yx_founds:
        for im in range(M):
            index = np.zeros(6,dtype=int)
            index[0] = im
            index[1] = iy 
            index[2] = ix
            index_mat = index_six_to_two(index,M,Ly,Lx)
            count_non_zeros_in_row[index_mat[0]] = 0

    # Create the sparse matrix
    Tsm_sm_sp = sparse.lil_matrix((M*Ly*Lx, M*Ly*Lx), dtype=np.double)

    # Fill the sparse matrix
    for im_new in range(M):
        
        left = [ (im_new, iy, clip(ix-1, 0, Lx-1), im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]
        left_act = [ (0, im_new, im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]
        
        for t, a in zip(left, left_act):
            indexes = index_six_to_two(t,M,Ly,Lx)
            if Tsm_sm_sp[indexes] == None :
                Tsm_sm_sp[indexes] =  p_a_mu_m_xy[a]
            else:
                Tsm_sm_sp[indexes] +=  p_a_mu_m_xy[a]
                
        right = [ (im_new, iy, clip(ix+1, 0, Lx-1), im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]
        right_act = [ (1, im_new, im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]

        for t, a in zip(right, right_act):
            indexes = index_six_to_two(t,M,Ly,Lx)
            if Tsm_sm_sp[indexes] == None :
                Tsm_sm_sp[indexes] =  p_a_mu_m_xy[a]
            else:
                Tsm_sm_sp[indexes] +=  p_a_mu_m_xy[a]

        up = [ (im_new, clip(iy+1, 0, Ly-1), ix,  im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]
        up_act = [ (2, im_new, im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]

        for t, a in zip(up, up_act):
            indexes = index_six_to_two(t,M,Ly,Lx)
            if Tsm_sm_sp[indexes] == None :
                Tsm_sm_sp[indexes] =  p_a_mu_m_xy[a]
            else:
                Tsm_sm_sp[indexes] +=  p_a_mu_m_xy[a]
        
        down = [ (im_new, clip(iy-1, 0, Ly-1), ix, im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]
        down_act = [ (3, im_new, im, iy, ix)   for ix in np.arange(Lx) for iy in np.arange(Ly) for im in np.arange(M)]

        for t, a in zip(down, down_act):
            indexes = index_six_to_two(t,M,Ly,Lx)
            if Tsm_sm_sp[indexes] == None :
                Tsm_sm_sp[indexes] =  p_a_mu_m_xy[a]
            else:
                Tsm_sm_sp[indexes] +=  p_a_mu_m_xy[a]

    # Delete the rows and columns that have been set to zero depending on the distance to the initial position
    yxs = it.product(np.arange(Ly), np.arange(Lx))
    yx_founds = it.filterfalse(lambda x: (x[0]-Ly0)**2 + (x[1]-Lx0)**2 > find_range**2, yxs)

    for yx_found in yx_founds:
        for im in range(M):
            # rows to set zero
            indexes = np.zeros(6,dtype=int)
            indexes[0] = indexes[3] = im
            indexes[1] = indexes[4] = yx_found[0]
            indexes[2] = indexes[5] = yx_found[1]
            indexes = index_six_to_two(indexes,M,Ly,Lx)
            # rows to set zero
            Tsm_sm_sp[indexes[0],:] = 0
            # columns to set zero
            Tsm_sm_sp[:,indexes[1]] = 0

    return Tsm_sm_sp

## Functions to the $\eta$ with different libraries

In [ ]:
def eta_petsc(Tsm_sm_matrix,gamma,M,Lx,Ly,rho0,ks_type='bcgs',ps_type='jacobi'):
    """
    Solve linear system for eta using PETSc
    """

    # Get the size of the matrix
    mat_size = M*Ly*Lx
    action_size = 4
    # Get the maximum number of non zeros values per row for Tsm_sm
    count_non_zeros_in_row = np.ones(mat_size,dtype=np.int32) * action_size * M
    count_non_zeros_in_row[0] -= M
    count_non_zeros_in_row[-1] -= M

    # print('Number of non-zero elements per row: ', count_non_zeros_in_row)

    # Create PETSc matrix
    A = PETSc.Mat()
    A.createAIJ([M*Lx*Ly,M*Lx*Ly], nnz=count_non_zeros_in_row)

    # Fill PETSc matrix
    non_zeros_Tsm = np.transpose(np.nonzero(Tsm_sm_matrix))
    for index in non_zeros_Tsm:
        A.setValues(index[0],index[1],Tsm_sm_matrix[index[0],index[1]])
    
    A.assemblyBegin()
    A.assemblyEnd()

    # eye matrix with PETSc
    ones = PETSc.Mat()
    ones.createAIJ(size=(M*Lx*Ly,M*Lx*Ly), nnz=1)
    for i in range(M*Lx*Ly):
        ones.setValues(i,i,1.0)

    ones.assemblyBegin()
    ones.assemblyEnd()

    # matrix_A =  I - gamma * T
    A.aypx(-gamma,ones)

    # print('Matrix A assembled', A.size)
    # print(A.getInfo())
    
    x, b =  A.createVecs()
    b.setValues(np.arange(rho0.shape[0],dtype=np.int32),rho0)
    b.assemble()

    # Start the solver
    ksp = PETSc.KSP().create()
    ksp.setOperators(A)
    ksp.setType(ks_type)
    ksp.getPC().setType(ps_type)
    ksp.setTolerances(rtol=1e-10)

    ksp.setUp()
    ksp.solve(b, x)

    ksp.destroy()
    A.destroy()
    ones.destroy()
    b.destroy()

    eta = x.getArray()
    x.destroy()

    return eta

In [ ]:

def eta_numpy(Tsm_sm_matrix,gamma,M,Lx,Ly,rho0):
    """
    Invert a matrix using numpy
    """
    to_invert = np.eye(M*Ly*Lx) - gamma * Tsm_sm_matrix
    inverted = np.linalg.inv(to_invert)
    new_eta = inverted @ rho0

    return new_eta

def eta_numpy_sol(Tsm_sm_matrix,gamma,M,Lx,Ly,rho0):
    """
    Solve linear system
    """
    mat_A = np.eye(M*Ly*Lx) - gamma * Tsm_sm_matrix
    new_eta = np.linalg.solve(mat_A,rho0)

    return new_eta

def eta_torch(Tsm_sm_matrix,gamma,M,Lx,Ly,rho0):
    """
    Invert a matrix using torch
    """
    Tsm_sm_matrix_torch = torch.from_numpy(Tsm_sm_matrix).type(dtype).to(device)

    rho0_torch = torch.from_numpy(rho0).type(dtype).to(device) 

    to_invert = torch.eye(M*Ly*Lx,dtype=dtype,device=device) - gamma * Tsm_sm_matrix_torch
    inverted = torch.torch.linalg.inv(to_invert)
    new_eta = inverted @ rho0_torch

    new_eta = new_eta.cpu().numpy()

    return new_eta

def eta_torch_sol(Tsm_sm_matrix,gamma,M,Lx,Ly,rho0):
    """
    Solve linear system using torch
    """
    Tsm_sm_matrix_torch = torch.from_numpy(Tsm_sm_matrix).type(dtype).to(device)

    rho0_torch = torch.from_numpy(rho0).type(dtype).to(device) 

    A_mat = torch.eye(M*Ly*Lx,dtype=dtype,device=device) - gamma * Tsm_sm_matrix_torch
    new_eta = torch.linalg.solve(A_mat,rho0_torch)

    new_eta = new_eta.cpu().numpy()

    return new_eta

def eta_scipy_sparce_solve(Tsm_sm_matrix,gamma,M,Lx,Ly,rho0):
    """
    Solve linear system using scipy sparce
    """
    Tsm_sm_matrix_sparce = sparse.csr_matrix(Tsm_sm_matrix)
    to_invert = sparse.eye(M*Ly*Lx) - gamma * Tsm_sm_matrix_sparce
    new_eta = sparse.linalg.spsolve(to_invert,rho0)

    return new_eta

def eta_cupy_sparse_solve(Tsm_sm_matrix,gamma,M,Lx,Ly,rho0):
    """
    Solve linear system using cupy sparse
    """
    Tsm_sm_matrix_cupy = cusparse.csr_matrix(Tsm_sm_matrix)
    to_invert = cusparse.eye(M*Ly*Lx) - gamma * Tsm_sm_matrix_cupy
    rho0_cupy = cp.asarray(rho0)
    new_eta_cupy = cusparse.linalg.spsolve(to_invert,rho0_cupy)
    new_eta = cp.asnumpy(new_eta_cupy)

    return new_eta


## Solve the problem

In [ ]:

def solve_eta(pi, PObs_lim, gamma, rho0, Lx, Ly, Lx0, Ly0, find_range,func_eta=eta_numpy, verbose=False):
    """
    This function should solve the following:
    --> New_eta = (1 - gamma T)^-1 rho
    """
    # O = Observations
    # M = Memory
    # A = Actions
    O, M, A = pi.shape
    # L = Dimension of the environment
    L = Lx * Ly
    new_eta = np.zeros(M*L)
    
    # PY has size ~ 10^5
    PY = PObs_lim.reshape(O, M, Ly, Lx)
    # PY has size ~ 10^2
    PAMU = pi.reshape(O, M, M, A//M)
    
    p_a_mu_m_xy = np.einsum( 'omyx, omna -> anmyx', PY, PAMU)
    # T [ s'm'  sm] = sum_a, mu p(s'm' | sm a mu) p(a mu | sm)
    #               = sum_a, mu p(s'm' | sm a mu) sum_y f(y | s) pi(a mu | y m)
    
    need_sparse = False
    if func_eta == eta_scipy_sparce_solve or func_eta == eta_petsc or func_eta == eta_cupy_sparse_solve:
        need_sparse = True
    # Tsm_sm has size ~ 10^5 x 10^5 or more
    if need_sparse:
        Tsm_sm_matrix = build_Tsm_sm_sparse(M,Lx,Ly,Lx0,Ly0,find_range,A//M,p_a_mu_m_xy)
    else:
        Tsm_sm_matrix = build_Tsm_sm_dense(M,Lx,Ly,Lx0,Ly0,find_range,A//M,p_a_mu_m_xy)

    if verbose:
        print("pi shape:",pi.shape)
        print("new_eta shape:",new_eta.shape)
        print("PY shape:",PY.shape, "PAMU shape:",PAMU.shape)
        print("p_a_mu_m_xy shape:",p_a_mu_m_xy.shape)

        if need_sparse:
            print("               Type of Tsm_sm_matrix:",type(Tsm_sm_matrix))
            print("number of non-zeros in Tsm_sm_matrix:",Tsm_sm_matrix.nnz)
            print("number of     zeros in Tsm_sm_matrix:",M*Lx*Ly*M*Lx*Ly - Tsm_sm_matrix.nnz)
            print("               opacity of the matrix: {:7.4f}".format(Tsm_sm_matrix.nnz/(M*Lx*Ly*M*Lx*Ly) * 100.0), "%")
            plt.imshow(Tsm_sm_matrix.toarray())
            plt.show()

        else :
            print("number of non-zeros in Tsm_sm_matrix:",np.count_nonzero(Tsm_sm_matrix != 0))
            print("number of     zeros in Tsm_sm_matrix:",np.count_nonzero(Tsm_sm_matrix == 0))
            print("               opacity of the matrix: {:7.4f}".format(np.count_nonzero(Tsm_sm_matrix != 0)/Tsm_sm_matrix.size * 100.0), "%") 
            plt.imshow(Tsm_sm_matrix)
            plt.show()

    new_eta = func_eta(Tsm_sm_matrix,gamma,M,Lx,Ly,rho0)
    
    return new_eta, Tsm_sm_matrix

## TEST

In [ ]:
M = 3 # O(10) 
O = 2 # 2/3
dim_factor = 2
Lx = 11 * dim_factor # O(100)-O(1000)
Ly = 15 * dim_factor # O(100)-O(1000)        
a_size = 4

# np.random.seed(33+33)

pi = softmax( np.random.rand(O,M,M*a_size), 2)

PObs_lim = np.random.rand(O, M*Lx*Ly)
PObs_lim[1] = 1-PObs_lim[0]

Lx0 = 5
Ly0 = 7
rho0 = np.random.rand(M*Lx*Ly)
rho0[Lx:] = 0
rho0 /= np.sum(rho0)
eta0 = np.random.rand(M*Lx*Ly)
gamma = 0.99
find_range = 2.1
tol = 1e-8

print("Solving with linear algebra")
inv_sol, T = solve_eta(pi, PObs_lim, gamma, rho0, Lx, Ly, Lx0, Ly0, find_range, verbose=True, func_eta=eta_scipy_sparce_solve)
print("Solving with iterative solver")
iter_sol = itsol(pi, PObs_lim, gamma, rho0, eta0, tol, Lx, Ly, Lx0, Ly0, find_range)


## Benchmarking of the solvers

In [ ]:
solvers =      [eta_numpy, eta_numpy_sol, eta_torch, eta_torch_sol, eta_scipy_sparce_solve, eta_cupy_sparse_solve, eta_petsc]
name_solvers = [  "Numpy",   "Numpy Sol",   "Torch",   "Torch Sol",         "Scipy Sparse",         "Cupy Sparse",   "Petsc"]

for solver, name_solver in zip(solvers, name_solvers):
    print("Method: {:>14}".format(name_solver), end=" ")
    %timeit solve_eta(pi, PObs_lim, gamma, rho0, Lx, Ly, Lx0, Ly0, find_range,func_eta=solver, verbose=False)
    # solve_eta(pi, PObs_lim, gamma, rho0, Lx, Ly, Lx0, Ly0, find_range,func_eta=solver, verbose=True)

print("\nIterative Solution", end=" ")

%timeit itsol(pi, PObs_lim, gamma, rho0, eta0, tol, Lx, Ly, Lx0, Ly0, find_range)

## Profiling

In [ ]:
# Profiler line by line
%load_ext line_profiler
# %lprun?

In [ ]:
for solver, name_solver in zip(solvers, name_solvers):
    print("Method: {:>14}".format(name_solver), end=" ")
    %lprun -f solve_eta solve_eta(pi, PObs_lim, gamma, rho0, Lx, Ly, Lx0, Ly0, find_range,func_eta=solver, verbose=False)

# %lprun -f solve_eta solve_eta(pi, PObs_lim, gamma, rho0, Lx, Ly, Lx0, Ly0, find_range)

In [ ]:
%lprun -f itsol  itsol(pi, PObs_lim, gamma, rho0, eta0, tol, Lx, Ly, Lx0, Ly0, find_range)

## Comparison of the results

In [ ]:
memory = 0

fig = plt.figure(figsize=(12,6))
plt.subplot(1,3,1)
plt.title("Inverse Solution")
plt.imshow(inv_sol.reshape(M,Ly,Lx)[memory])
plt.colorbar()
plt.subplot(1,3,2)
plt.title("Iterative Solution")
plt.imshow(iter_sol.reshape(M,Ly,Lx)[memory])
plt.colorbar()
plt.subplot(1,3,3)
plt.title("Difference")
plt.imshow(np.abs(iter_sol.reshape(M,Ly,Lx)[memory]-inv_sol.reshape(M,Ly,Lx)[memory]))
plt.colorbar()
plt.show()



Test if the solution of the different solvers are correct, using 
$$\mathbf{\rho} - (\mathbf{1} - \gamma \mathbf{T}) \mathbf{\eta} = 0 $$ 

In [ ]:
print("Shape     iter_sol:", iter_sol.shape[0],
          "max: {:.4f} min: {:.4f}".format(np.max(iter_sol), np.min(iter_sol)))

for solver, name_solver in zip(solvers, name_solvers):
    inv_sol, T = solve_eta(pi, PObs_lim, gamma, rho0, Lx, Ly, Lx0, Ly0, find_range,func_eta=solver)
    print("Shape {:>12}: {:} max: {:.4f} min: {:.4f}".format(name_solver, inv_sol.shape[0], np.max(inv_sol), np.min(inv_sol)), end=" ")  
    print("|| Passing test of solver:",solution_test(inv_sol,T,rho0,gamma), end=" ")
    print("|| Diff from iter_sol: {:.8f}".format(np.sum(np.abs(iter_sol-inv_sol))))
print("-"*50)
print("Diff between solvers")        
for i in range(len(solvers)):
    inv_sol_i, T_i = solve_eta(pi, PObs_lim, gamma, rho0, Lx, Ly, Lx0, Ly0, find_range,func_eta=solvers[i])
    for j in range(i+1,len(solvers)):
        inv_sol_j, T_j = solve_eta(pi, PObs_lim, gamma, rho0, Lx, Ly, Lx0, Ly0, find_range,func_eta=solvers[j])
        print("Difference between {:>12} and {:>12}: {:.5e}".format(name_solvers[i], name_solvers[j], np.sum(np.abs(inv_sol_i-inv_sol_j)))) 


Compare the solution of each solver with the iterative solution

In [ ]:

for solver, name_solver in zip(solvers, name_solvers):
    print("Test: {:>12}".format(name_solver))
    inv_sol, T = solve_eta(pi, PObs_lim, gamma, rho0, Lx, Ly, Lx0, Ly0, find_range,func_eta=solver)
    print("Shape inv_sol:", inv_sol.shape, "max:", np.max(inv_sol), "min:", np.min(inv_sol))
    print("Passing test of solver",solution_test(inv_sol,T,rho0,gamma,plot=True))
    
    fig = plt.figure(figsize=(12,6))
    plt.subplot(1,3,1)
    plt.title(name_solver)
    plt.imshow(inv_sol.reshape(M,Ly,Lx)[memory])
    plt.colorbar()
    plt.subplot(1,3,2)
    plt.title("Iterative Solution")
    plt.imshow(iter_sol.reshape(M,Ly,Lx)[memory])
    plt.colorbar()
    plt.subplot(1,3,3)
    plt.title("Difference")
    plt.imshow(np.abs(iter_sol.reshape(M,Ly,Lx)[memory]-inv_sol.reshape(M,Ly,Lx)[memory]))
    plt.colorbar()
    plt.show()


Compare all the solutions generated in all permutations 

In [ ]:
    
for i in range(len(solvers)):
    for j in range(i+1,len(solvers)):
        inv_sol_i, T_i = solve_eta(pi, PObs_lim, gamma, rho0, Lx, Ly, Lx0, Ly0, find_range,func_eta=solvers[i])
        inv_sol_j, T_j = solve_eta(pi, PObs_lim, gamma, rho0, Lx, Ly, Lx0, Ly0, find_range,func_eta=solvers[j])
        print("Difference between", name_solvers[i], "and", name_solvers[j], np.sum(np.abs(inv_sol_i-inv_sol_j)))

        fig = plt.figure(figsize=(12,6))
        plt.subplot(1,3,1)
        plt.title(name_solvers[i])
        plt.imshow(inv_sol_i.reshape(M,Ly,Lx)[memory])  
        plt.colorbar()
        plt.subplot(1,3,2)
        plt.title(name_solvers[j])
        plt.imshow(inv_sol_j.reshape(M,Ly,Lx)[memory])
        plt.colorbar()
        plt.subplot(1,3,3)
        plt.title("Difference")
        plt.imshow(np.abs(inv_sol_i.reshape(M,Ly,Lx)[memory]-inv_sol_j.reshape(M,Ly,Lx)[memory]))
        plt.colorbar()
        plt.show()
        
